In [1]:
import os
import csv
import matplotlib.image as mpimg
import cv2
import numpy as np
import sklearn
import keras
from sklearn.utils import shuffle
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D,Lambda,Cropping2D
from keras.optimizers import Adadelta,SGD,Adam
from keras.losses import categorical_crossentropy
from keras.callbacks import LearningRateScheduler, ModelCheckpoint







/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.
/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary in

In [2]:
samples = []
correction=0.2
with open('./data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)
del samples[0]

from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(samples, test_size=0.2)




def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                path='./data/'
                center_image=mpimg.imread(path+batch_sample[0])
                #left_image=cv2.imread(path+batch_sample[1][1:])
                #right_image=cv2.imread(path+batch_sample[2][1:])
                images.append(center_image)
                #images.extend([center_image,left_image,right_image])
                
                center_angle=float(batch_sample[3])
                #left_angle=center_angle+correction
                #right_angle=center_angle-correction
                
                #angles.extend(center_angle,left_angle,right_angle])
                angles.append(center_angle)

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            
            
            yield shuffle(X_train, y_train)

In [3]:
# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)
epochs=5
#batch_size=32

In [ ]:
model=Sequential()
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((50,20), (0,0))))
model.add(Conv2D(32, (3, 3), padding='SAME',activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))

model.add(Conv2D(64, (3, 3), padding='SAME',activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))

model.add(Conv2D(128, (3, 3), padding='SAME',activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='softmax'))

model.compile(loss='mse', optimizer='adam')
model.fit_generator(train_generator, steps_per_epoch= len(train_samples),
validation_data=validation_generator, validation_steps=len(validation_samples), epochs=3, verbose = 1)

Epoch 1/3


In [ ]:
model.save('model.h5')